# Power Flow Analysis

Power flow analysis, also known as load flow, is the foundation of power system studies. It calculates the steady-state operating point of a power system by solving a set of nonlinear algebraic equations that represent power balance at each bus. The results include voltage magnitudes and angles at all buses, as well as active and reactive power flows on all branches.

This tutorial covers running power flow in ANDES, accessing and interpreting results, configuring solver options, and troubleshooting convergence issues.

## Setup

We begin by importing ANDES and configuring the logger. The logger controls what information is displayed during execution, with level 20 (INFO) being appropriate for normal use.

In [ ]:
import andes

andes.config_logger(stream_level=20)

## Running Power Flow

To run power flow, you first load a case file to create a System object, then call the `run()` method on the `PFlow` routine. ANDES implements the Newton-Raphson method, which iteratively solves the power balance equations until the mismatch (residual) falls below the convergence tolerance.

For this tutorial, we use the IEEE 14-bus test system, a standard benchmark that contains 14 buses, 5 generators, and 20 branches. This system is small enough to examine results easily while being realistic enough to demonstrate typical power flow behavior.

In [ ]:
ss = andes.load(andes.get_case('ieee14/ieee14.json'))
ss.PFlow.run()

After the solver completes, you can check whether the solution converged and how many iterations were required. A well-conditioned power flow typically converges in 3-6 iterations. If the iteration count is high or convergence fails, see the Troubleshooting section below.

In [ ]:
print(f"Converged: {ss.PFlow.converged}")
print(f"Iterations: {ss.PFlow.niter}")

### Command Line Alternative

Power flow can also be executed from the command line without writing Python code. This is useful for quick analyses or batch processing. The following command runs power flow on a case file:

```bash
andes run ieee14.json
```

The CLI automatically generates output files containing the results.

## Accessing Results

After power flow converges, results are stored in the model objects that make up the System. Each model (such as `Bus`, `Line`, `PV`) contains variables whose solved values can be accessed through the `.v` attribute. Understanding how to navigate this structure is essential for extracting the information you need.

### Bus Results

Bus voltages are the primary output of power flow analysis. The voltage magnitude is stored in `ss.Bus.v` and the voltage angle in `ss.Bus.a`. To access the numerical values as NumPy arrays, append `.v` to get the value array.

In [ ]:
# Voltage magnitudes (per unit)
ss.Bus.v.v

In [ ]:
# Voltage angles (radians)
ss.Bus.a.v

For a more comprehensive view, you can retrieve all bus parameters and variables as a pandas DataFrame using `cache.df`. This is particularly useful for inspecting multiple quantities at once or exporting results to other tools.

In [ ]:
ss.Bus.cache.df

### Generator Results

Generator outputs are distributed across different model types depending on their control mode. In power flow, `PV` buses represent generators with fixed voltage magnitude and active power output, while `Slack` (or swing) buses balance the system by absorbing any mismatch. The solved reactive power output `q` is determined by the power flow solution.

In [ ]:
print("PV generators - Active power (pu):", ss.PV.p.v)
print("Slack generator - Active power (pu):", ss.Slack.p.v)

In [ ]:
print("PV generators - Reactive power (pu):", ss.PV.q.v)
print("Slack generator - Reactive power (pu):", ss.Slack.q.v)

### Line Flows

Branch power flows indicate how power is transmitted through the network. For each line, `a1` represents the active power flow at the from-bus end, and `a2` represents the active power flow at the to-bus end. The difference between these values accounts for transmission losses.

In [ ]:
# Show flows for first 5 lines
print("From-bus active power (pu):", ss.Line.a1.v[:5])
print("To-bus active power (pu):", ss.Line.a2.v[:5])

## Configuration

The power flow solver behavior can be customized through configuration options accessible via `ss.PFlow.config`. These settings control convergence criteria, iteration limits, and solver behavior. Viewing the current configuration shows all available options and their values.

In [ ]:
ss.PFlow.config

The most commonly adjusted options are summarized below:

| Option | Default | Description |
|--------|---------|-------------|
| `max_iter` | 25 | Maximum number of Newton-Raphson iterations |
| `tol` | 1e-6 | Convergence tolerance for power mismatch |
| `method` | 'NR' | Solution method (NR = Newton-Raphson) |
| `init_tds` | 0 | Whether to initialize TDS models after power flow |

To modify these settings, assign new values directly to the config attributes. Changes take effect on the next `run()` call.

In [ ]:
ss.PFlow.config.max_iter = 50
ss.PFlow.config.tol = 1e-8

For persistent configuration that applies to all ANDES sessions, you can edit the configuration file located at `~/.andes/andes.conf`. Settings in this file are loaded automatically when ANDES starts.

```ini
[PFlow]
max_iter = 50
tol = 1e-8
```

## Troubleshooting

### Non-Convergence

Power flow convergence failures are typically caused by one of three issues:

1. **Data errors**: Zero impedance lines create singular Jacobian matrices, and isolated buses have no valid solution. Check your network topology for disconnected components.

2. **Infeasible operating point**: If total load exceeds generation capacity, or if reactive power requirements cannot be met by available sources, no solution exists. Verify that generation and load are balanced.

3. **Poor initial guess**: The Newton-Raphson method requires a reasonable starting point. For very stressed systems, the flat start (1.0 pu voltage, 0 angle) may be too far from the solution.

When convergence fails, try the following debugging steps:

In [ ]:
# Allow more iterations for difficult cases
ss.PFlow.config.max_iter = 100

# Enable PV-to-PQ conversion when generators hit reactive limits
ss.PV.config.pv2pq = 1

### Viewing Iteration Progress

To diagnose convergence issues, you can enable debug-level logging which displays the residual magnitude at each iteration. A healthy convergence shows the residual decreasing rapidly (quadratic convergence is characteristic of Newton-Raphson). If the residual oscillates or decreases slowly, there may be numerical issues with the case data.

In [ ]:
andes.config_logger(stream_level=10)  # DEBUG level

ss_debug = andes.load(andes.get_case('ieee14/ieee14.json'))
ss_debug.PFlow.run()

The output shows the residual norm at each iteration:
```
0: |F(x)| = 14.9282832
1: |F(x)| = 3.608627841
2: |F(x)| = 0.1701107882
...
```

Quadratic convergence means that each iteration roughly squares the number of correct digits, so you should see the residual drop by several orders of magnitude in just a few iterations.

## Power Flow Report

After a successful power flow, ANDES generates a text report file containing formatted results. This report includes system statistics, bus data (voltage, angle, injections), and branch data (flows, losses). The report path can be accessed through the `files` attribute.

In [ ]:
ss.files.txt

The report is organized into sections:
1. **System statistics**: Total generation, load, and losses
2. **Bus data**: Voltage magnitude and angle, P/Q injection at each bus
3. **Line data**: Active and reactive power flows, losses on each branch
4. **Other algebraic variables**: Additional solved quantities from device models

## Example: Formatted Bus Voltage Display

The following example demonstrates how to iterate through bus results and display them in a readable format. This pattern is useful for creating custom reports or validating results against other tools.

In [ ]:
import math

print("Bus Voltage Results")
print("-" * 40)
for name, v, a in zip(ss.Bus.name.v, ss.Bus.v.v, ss.Bus.a.v):
    print(f"Bus {name:>4}: V = {v:.4f} pu, θ = {math.degrees(a):>7.2f}°")

## Cleanup

Remove output files generated during this tutorial.

In [ ]:
!andes misc -C

## Next Steps

With power flow complete, you can proceed to dynamic analysis:

- {doc}`04-time-domain` - Time-domain simulation with disturbances
- {doc}`05-data-and-formats` - Loading different file formats and modifying parameters